In [102]:
###1st step
import pandas as pd
import numpy as np

import cx_Oracle as ora
import pyodbc

import datetime

In [103]:
piloto_anterior = pd.read_csv('02_piloto_whitelist_etiquetado.csv',sep='\t') ### Datos de los pilotos anteriores
predict_base = pd.read_csv('prp_complete_predict_wallet211115.csv') ### Solo datos de predicción
real_base = pd.read_csv('prp_complete_real_wallet211115.csv') ### Con los datos reales

n01_pilot_prob = 0.0000001
n02_pilot_prob = 0.0000010
n03_pilot_prob = 0.0000100

predict_base.loc[predict_base['p1'] < n02_pilot_prob, 'predict2'] = 1
predict_base.loc[predict_base['p1'] >=n02_pilot_prob, 'predict2'] = 0

predict_base['predict2'] = predict_base['predict2'].astype('category')

joined_base = predict_base.merge(real_base, left_on='Unnamed: 0', right_on='Unnamed: 0', how='inner', indicator=False)
joined_base = joined_base.merge(piloto_anterior, left_on='msisdn', right_on='Telefono', how='outer', indicator=True)

joined_base['Telefono'] = np.where(joined_base['Telefono'].isnull(), joined_base['msisdn'], joined_base['Telefono'])
joined_base['msisdn'] = np.where(joined_base['msisdn'].isnull(), joined_base['Telefono'], joined_base['msisdn'])
joined_base['predict2'] = np.where(joined_base['predict2'].isnull(), 1, joined_base['predict2'])

joined_base.loc[(joined_base['Base'].isnull()) & (joined_base['predict2']==1) & (joined_base['potential_bancarizado']==1), 'Base'] = 'Predictor3'

In [107]:
#joined_base.count()
#joined_base[joined_base['predict2']==1].groupby('Base').msisdn.count()
#joined_base.groupby(['Base','predict2']).msisdn.count()
joined_base[joined_base['Base'].notnull()].groupby('Base').msisdn.count()

##EXPORTING BASE TO CSV FOR SHARING WITH RA Team:
#joined_base[joined_base['Base'].notnull()][['Base','Telefono']].drop_duplicates(subset=['Telefono']).sort_values(by=['Base']).to_csv('03_piloto_whitelist_etiquetado.csv', index=False)


#joined_base[joined_base['predict2']==1].msisdn.count()
#joined_base.head()
#joined_base[(joined_base['predict2']==0) & (joined_base['Base']=='Predictor3')].count()
#joined_base[(joined_base['predict2']==1)].head()
#joined_base[(joined_base['predict2'].isnull())].groupby('Base').msisdn.count()

Base
Empleados      1801
Inicial       23139
Predictor     20212
Predictor2     8514
Predictor3     9657
Name: msisdn, dtype: int64

In [132]:
###2nd step
##### READING SUBS DATA FROM DB:

cxn = pyodbc.connect("DSN=hivedwh",autocommit=True)

dataList = []

with open('query_subs_db.sql', 'r') as file:
    query = file.read().replace('\n', '')

i = 1
    
for chunk in pd.read_sql_query(sql=query,con=cxn, chunksize = 200000):
    dataList.append(chunk)
    i = i+1

print("######### Loading Data Frame #########")

data = pd.concat(dataList, ignore_index=True)
print(len(data))
print("######### Fin #########")

dataList = []

#data.telefono = data.telefono.astype(int)

#### Reading data from model output:

piloto_anterior = pd.read_csv('02_piloto_whitelist_etiquetado.csv',sep='\t') ### Datos de los pilotos anteriores
predict_base = pd.read_csv('prp_complete_predict_wallet211115.csv') ### Solo datos de predicción
real_base = pd.read_csv('prp_complete_real_wallet211115.csv') ### Con los datos reales

n01_pilot_prob = 0.0000001
n02_pilot_prob = 0.0000010
n03_pilot_prob = 0.0000100

predict_base.loc[predict_base['p1'] < n02_pilot_prob, 'predict2'] = 1
predict_base.loc[predict_base['p1'] >=n02_pilot_prob, 'predict2'] = 0

predict_base['predict2'] = predict_base['predict2'].astype('category')

joined_base = predict_base.merge(real_base, left_on='Unnamed: 0', right_on='Unnamed: 0', how='inner', indicator=False)
joined_base = joined_base.merge(piloto_anterior, left_on='msisdn', right_on='Telefono', how='outer', indicator=True)

joined_base['Telefono'] = np.where(joined_base['Telefono'].isnull(), joined_base['msisdn'], joined_base['Telefono'])
joined_base['msisdn'] = np.where(joined_base['msisdn'].isnull(), joined_base['Telefono'], joined_base['msisdn'])
joined_base['predict2'] = np.where(joined_base['predict2'].isnull(), 1, joined_base['predict2'])

joined_base.loc[(joined_base['Base'].isnull()) & (joined_base['predict2']==1) & (joined_base['potential_bancarizado']==1), 'Base'] = 'Predictor3'

#predict_base.head()
#real_base.head()
#joined_base.head()

#data.head()

Error: ('HY000', '[HY000] [Cloudera][DriverSupport] (1110) Unexpected response received from server. Please ensure the server host and port specified for the connection are correct and confirm if SSL should be enabled for the connection. (1110) (SQLDriverConnect); [HY000] [Cloudera][DriverSupport] (1110) Unexpected response received from server. Please ensure the server host and port specified for the connection are correct and confirm if SSL should be enabled for the connection. (1110)')

In [131]:
# buscando la base final para subir a Oracle:
joined_base.loc[joined_base['Base'].isnull(),'Base'] = 'NA'

joined_base.flag = 0
joined_base.loc[(joined_base['Base']=='Empleados') | (joined_base['Base']=='Inicial') | (joined_base['Base']=='Predictor') | (joined_base['Base']=='Predictor2') | (joined_base['Base']=='Predictor3'),'flag'] = 1
#joined_base[(joined_base['Base']=='Empleados') | (joined_base['Base']=='Inicial') | (joined_base['Base']=='Predictor') | (joined_base['Base']=='Predictor2') | (joined_base['Base']=='Predictor3')].drop_duplicates(subset=['msisdn']).groupby('flag').msisdn.count()
#joined_base.drop_duplicates(subset=['msisdn']).groupby(['Base','predict2']).msisdn.count()
joined_base.drop_duplicates(subset=['msisdn']).groupby(['Base','flag']).msisdn.count()
#joined_base[(joined_base['Base']=='Empleados') | (joined_base['Base']=='Inicial') | (joined_base['Base']=='Predictor') | (joined_base['Base']=='Predictor2') | (joined_base['Base']=='Predictor3')].drop_duplicates(subset=['msisdn']).count()
#joined_base.loc[joined_base['Base'].isnull(),'flag'].head()

Base        flag
Empleados   1         1801
Inicial     1        23073
NA          0       428749
Predictor   1        20193
Predictor2  1         8513
Predictor3  1         9653
Name: msisdn, dtype: int64

In [104]:
###3rd and last step

#### FILLING ALL Bases with no Base
joined_base.loc[joined_base['Base'].isnull(),'Base'] = 'NA'

#### Refactoring flag, putting 0 in all using, in the next step I will be putting 1 for the bases
joined_base.flag = 0
#### Putting 1 to users in the choosen bases:
joined_base.loc[(joined_base['Base']=='Empleados') | (joined_base['Base']=='Inicial') | (joined_base['Base']=='Predictor') | (joined_base['Base']=='Predictor2') | (joined_base['Base']=='Predictor3'),'flag'] = 1

joined_base = joined_base.drop_duplicates(subset=['telefono'])

###### BEGIN EXPORTING:

user = 'BI'
passw = 'dwhprodbi20$'
server = 'srvdwhdb.celtel.net:1521/dwh.celtel.net'

con = ora.connect(user, passw, server)
cur = con.cursor()

a=datetime.datetime.now()

cur.execute('delete from bi.cc_cashin_model_output')

query = "insert into bi.cc_cashin_model_output(telefono, flag) values(:1,:2)"

cur.executemany(query,joined_base[['telefono','predict2']].head(6000000).values.tolist())

#### Test lines:
cur.execute('delete from bi.cc_cashin_model_output where telefono = 97047014')
cur.execute('insert into bi.cc_cashin_model_output(telefono, flag) values(97047014,0)')

cur.execute('delete from bi.cc_cashin_model_output where telefono = 98284795')
cur.execute('insert into bi.cc_cashin_model_output(telefono, flag) values(98284795,1)')

con.commit()

con.close()

b=datetime.datetime.now()

print('Time Elapsed: {}'.format(b-a))

Time Elapsed: 0:01:38.192660
